# Confidence and Choice Visualizer

Run DRAM_classify_blobs 10 times (use runDRAM_classify_blobs.sh), then use ClassificationDistributions.ipynb to output confidence_one_run for each run, and set them as the confidence for each run in confidence_all_runs.

In [ ]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from analysis import classify_image, glimpses, read_n, classify_imgs2

In [ ]:
num_runs = 1#10
iter_list = [i for i in range(0, 100000, 10000)]#[250, 500, 1000, 2000, 4000, 8000, 16000, 32000, 64000, 125000, 250000, 500000]
num_iters = len(iter_list)
z_size = 9

confidence_all_runs = np.zeros([num_runs, num_iters, z_size, z_size])
choice_all_runs = np.zeros([num_runs, num_iters, z_size, z_size])

In [ ]:
def fill_matrix(path, iteration):
    """Fill the confidence and choice matrices for one run at one iteration."""
    
    data = None
    num_imgs = 9000
    imgs_data = classify_imgs2(iteration, True, num_imgs, path=path)
    
    max_blobs = 9
    min_blobs = 1
    z_size = max_blobs - min_blobs + 1
    confidence_one_run = np.zeros([z_size, z_size])
    choice_one_run = np.zeros([z_size, z_size])


    for nb in range(z_size):
        num_blobs = nb + 1
        confidence_hist = np.zeros(z_size)
        choice_hist = np.zeros(z_size)
        value_counts = np.zeros(z_size)
        values_sum = 0
        sqr_sum = 0
        num_imgs_with_num_blobs = 0

        for idx, data in enumerate(imgs_data):

            if data["label"][(num_blobs - min_blobs)] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1
                
                max_glimpse = 9
                min_glimpse = 0
                glimpses = 10
                
                for i in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
                    # Histogram of softmaxes
                    confidence_hist += data["classifications"][i][0] / glimpses
                    
                    # Histogram of choices
                    choice = np.argmax(data["classifications"][i][0])
                    choice_list = [0] * z_size
                    choice_list[choice] = 1 / glimpses
                    choice_hist += choice_list
        
        confidence_hist = confidence_hist / num_imgs_with_num_blobs
        confidence_one_run[nb] = confidence_hist.tolist()
        
        choice_hist = choice_hist / num_imgs_with_num_blobs
        choice_one_run[nb] = choice_hist.tolist()
        
    print("Confidence (One Run): ")
    print(confidence_one_run.tolist())
    print("Choice (One Run): ")
    print(choice_one_run.tolist())
    return confidence_one_run, choice_one_run

In [ ]:
model = "DRAM_classify_blobs"
for run in range(1, num_runs + 1):
#     path = 'model_runs/' + model + '_run_' + str(run)
    path = 'model_runs/DRAM_test_square'
    for i, iteration in enumerate(iter_list):
        confidence_all_runs[0, i], choice_all_runs[0, i] = fill_matrix(path, iteration)

In [ ]:
# for iteration in [250]:
#     iter_idx = iter_list.index(iteration)
#     confidence_all_runs[0, iter_idx] = np.array([[0.860760747172113, 0.1277467767290509, 0.010657337000120606, 0.0007285194326832247, 6.787944351642578e-05, 2.9971955746236867e-05, 5.145775413146485e-06, 7.464326014394856e-07, 2.878851030397534e-06], [0.20300415922482953, 0.5184953242166103, 0.20827241095635282, 0.05231078128653731, 0.012505148318335242, 0.004580371038418926, 0.0006180559509162963, 0.00010845492016917556, 0.00010529498801776148], [0.017868306261641757, 0.3537408285439692, 0.34259592842755954, 0.1658108561628833, 0.07394838094437096, 0.037945567332524535, 0.0058128362575559, 0.0015080746380952204, 0.0007692225027735955], [0.0009324774553806627, 0.13202229804306542, 0.274546225576406, 0.23018302718121686, 0.16847032805212483, 0.14204805273613272, 0.030567274960408255, 0.014227405173598084, 0.007002910586340956], [0.0003520191396409994, 0.04111785734034918, 0.15463699939020453, 0.2050167920370368, 0.2134346593449809, 0.24783463010188098, 0.06838748303711407, 0.04378451394166884, 0.02543504510230652], [5.000795444690018e-06, 0.01044296698407568, 0.06641280716187803, 0.13162901687612752, 0.18505500357876825, 0.31054571315684004, 0.1161021573583348, 0.10390606812600003, 0.07590126688454743], [1.3284532939238279e-06, 0.0023274134316703856, 0.021358992187837482, 0.06635526982578113, 0.1281593052150456, 0.29800758347616646, 0.14616071119542176, 0.17792251735337322, 0.15970687958101304], [8.615321391135292e-08, 0.000515853959362386, 0.006578610663797049, 0.02975448711197077, 0.0733464273270929, 0.2329618283142827, 0.14965183055811213, 0.2368133880313326, 0.2703774875925387], [1.8369941338283736e-08, 0.00014249697360372787, 0.002083101829095979, 0.012640375018897986, 0.036423716315071956, 0.1483100470611913, 0.12314310943966848, 0.268005807373178, 0.4092513283179578]])
#     choice_all_runs[0, iter_idx] = np.array([[0.9582000000001493, 0.04180000000000032, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.2021999999999928, 0.6375000000000764, 0.15339999999999557, 0.006099999999999994, 0.0002, 0.0006, 0.0, 0.0, 0.0], [0.008399999999999986, 0.3843000000000189, 0.4561000000000352, 0.10799999999999814, 0.029400000000000148, 0.013699999999999967, 0.0001, 0.0, 0.0], [0.0004, 0.08969999999999918, 0.39560000000002143, 0.2231999999999916, 0.10709999999999818, 0.18179999999999397, 0.00030000000000000003, 0.0013, 0.0006], [0.0002, 0.013699999999999967, 0.18319999999999387, 0.21249999999999222, 0.1493999999999958, 0.41790000000002653, 0.003900000000000002, 0.011399999999999976, 0.007799999999999988], [0.0, 0.0007999999999999999, 0.056500000000000536, 0.10939999999999805, 0.10209999999999847, 0.6116000000000705, 0.008899999999999984, 0.05640000000000053, 0.0543000000000005], [0.0, 0.0007999999999999999, 0.007599999999999989, 0.03770000000000027, 0.05780000000000055, 0.5834000000000641, 0.013599999999999968, 0.13329999999999673, 0.16579999999999487], [0.0, 0.0002, 0.0014000000000000002, 0.007499999999999989, 0.018799999999999997, 0.41000000000002473, 0.015299999999999961, 0.18219999999999392, 0.3646000000000144], [0.0, 0.0, 0.0004, 0.0022000000000000006, 0.005999999999999995, 0.21529999999999205, 0.012399999999999972, 0.15059999999999574, 0.6131000000000708]])

In [ ]:
def adj_all_runs(all_runs_matrix):
    """Adjust all the matrices so tick marks start with 1."""
    
    new_all_runs_matrix = np.zeros([num_runs, num_iters, z_size + 1, z_size + 1])
    for m, matrix in enumerate(all_runs_matrix):
        for i in range(num_iters):
            new_all_runs_matrix[m, i] = adj_matrix(matrix[i])
    return new_all_runs_matrix


def adj_matrix(matrix):
    """Adjust the matrix so tick marks start with 1."""
    
    a = np.zeros([1, z_size])
    temp = np.vstack((a, matrix))
    b = np.zeros([z_size + 1, 1])
    new_matrix = np.hstack((b, temp))
    return new_matrix


confidence_all_runs_adj = adj_all_runs(confidence_all_runs)
choice_all_runs_adj = adj_all_runs(choice_all_runs)


In [ ]:
def plot_confidence(run, iter_idx, it):
    """Plot the confidence heatmap."""
    
    matrix = confidence_all_runs_adj[run, iter_idx]
    plot_title = "Confidence at Run %d after %d Iterations" % (run + 1, it)
    plot_heatmap(matrix, plot_title)


def plot_choice(run, iter_idx, it):
    """Plot the choice heatmap."""
    
    matrix = choice_all_runs_adj[run, iter_idx]
    plot_title = "Choice at Run %d after %d Iterations" % (run + 1, it)
    plot_heatmap(matrix, plot_title)
    
    
def plot_heatmap(matrix, plot_title):
    """Plot heatmap."""
    
    data = [go.Heatmap(
        z=matrix,
        colorscale="Viridis"
    )]

    layout = go.Layout(
        title=plot_title,
        yaxis=dict(
            range=[9.5, 0.5],
            title="True Class",
            dtick=1
        ),
        xaxis=dict(
            range=[0.5, 9.5],
            title="Predicted Class",
            dtick=1
        ),
        width=500,
        height=500
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [ ]:
for run in range(num_runs):
#     path = 'model_runs/' + model + '_run_' + str(run)
    path = 'model_runs/DRAM_test_square'
    for i, it in enumerate(iter_list):
        plot_confidence(run, i, it)
        plot_choice(run, i, it)